In [67]:
import cv2
import numpy as np
from scipy import ndimage as ndi
from skimage.morphology import square,dilation
from skimage.io import imshow
import time
import copy

In [72]:
class HoughTransform:    
    def __init__(self,threshold = 40):
        self.url = '../video/my_video.h264'
        self.cap = cv2.VideoCapture(self.url)
        self.maskR = False
        self.ret = 0
        self.frame = 0
        self.slope_right = 0
        self.slope_left = 0
        self.slope = 0
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.threshold = threshold
    
    def region_of_interest(self,img, vertices):   
        mask = np.zeros_like(img)
        match_mask_color = (255,)
        cv2.fillPoly(mask, vertices, 255)
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image
    
    def findSlope(self,start,end,height):
        x1 = start[0]
        x2 = end[0]
        y1 = height-start[1]
        y2 = height-end[1]
        if (x2 - x1) != 0:
            m = (y2-y1)/(x2-x1)
            return m 

    def avgSlope(self,li):             
        x1 = round(sum(li[:,0])/len(li)).astype('int')
        x2 = round(sum(li[:,1])/len(li)).astype('int')
        y1 = round(sum(li[:,2])/len(li)).astype('int')
        y2 = round(sum(li[:,3])/len(li)).astype('int')
        return x1,x2,y1,y2

    def debug(self,debug = False):
        if (debug):            
            #cv2.putText(self.frame,str(self.slope_right),(100,25), self.font, 0.5,(255,255,255),2,cv2.LINE_AA)
            print('slope_right :', self.slope_right)            
            print('slope_left :', self.slope_left)            
            print('slope :', self.slope)
            print()
            
    def run(self,frame=None,fromvideo=False,debugs = False):
        if(fromvideo):
            while self.cap.isOpened():
                self.ret, self.frame = self.cap.read()
                if not self.ret:
                    print("Can't receive frame (stream end?). Exiting ...")
                    break                
                if cv2.waitKey(1) == ord('q'):
                    break
                self.corerun()                    
                self.debug(debug = debugs)
            self.cap.release()
            cv2.destroyAllWindows()
        else:
            self.frame = frame
            while cv2.waitKey(1) != ord('q'):                
                self.corerun()  
                self.debug(debug = debugs)
                return float(self.slope),self.maskR
            self.cap.release()
            cv2.destroyAllWindows()
            
    def corerun(self):
        self.maskR = False
        self.frame = cv2.resize(self.frame,(100, 75)) 
        height=self.frame.shape[0]
        width=self.frame.shape[1]
        
        self.frame=cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)    
        lowerR=np.array([150,100,100])
        upperR=np.array([255,190,190])    
        maskR = cv2.inRange(self.frame, lowerR, upperR)
        
        countR=0
        for i in range(width):            
            if maskR[30][i] == 255 or maskR[31][i] == 255 or maskR[32][i] == 255:
                countR+=1
#                 print(countR)
            if countR > 10:
                self.maskR = True
                print('stop')
                break
        
        # color range
        lowerY=np.array([200,200,0])
        upperY=np.array([255,255,204])    
        maskY = cv2.inRange(self.frame, lowerY, upperY)

        lowerW=np.array([170,190,190])
        upperW=np.array([255,255,255])    
        maskW = cv2.inRange(self.frame, lowerW, upperW)

        mask = maskY + maskW
        
        # crop image
        region_of_interest_vertices = [   
            (0, height),
            (0, (height/3)+10),
            (width, (height/3)+10),
            (width, height)]
        crop = self.region_of_interest(mask,np.array([region_of_interest_vertices], np.int32))
        crop = dilation(crop, square(3))
        lines = cv2.HoughLinesP(crop,rho = 1,theta = 1*np.pi/180,threshold = 40,minLineLength = 10,maxLineGap = 250)
    
        dframe = copy.deepcopy(self.frame) 
        l=[]
        r=[]        
        # Draw lines on the image
        if(lines is not None) :
            for line in lines:  
                x1, y1, x2, y2 = line[0]
                if(self.findSlope((x1,y1),(x2,y2),height)):            
                    if(x1 < int(width/2)):
                        l.append([x1,x2,y1,y2,self.findSlope((x1,y1),(x2,y2),height)])
                    else:
                        r.append([x1,x2,y1,y2,self.findSlope((x1,y1),(x2,y2),height)])
                        
            l=np.array(l)
            r=np.array(r)

            if(len(l)):                
                x1,x2,y1,y2 = self.avgSlope(l)
                self.slope_left = self.findSlope((x1,y1),(x2,y2),height)
                #print(x1,x2,y1,y2,'slope_left',slope_left)
                cv2.line(dframe, (x1, y1), (x2, y2), (255, 0, 0), 3)

            if(len(r)):                
                x1,x2,y1,y2 = self.avgSlope(r)
                self.slope_right = self.findSlope((x1,y1),(x2,y2),height)
                #print(x1,x2,y1,y2,'slope_right',slope_right)
                cv2.line(dframe, (x1, y1), (x2, y2), (0, 0, 255), 3)

            self.slope = round(self.slope_right + self.slope_left,3)
            if(self.slope > 2):
                self.slope = 2
            elif(self.slope < -2):
                self.slope = -2  
            print('slope',self.slope)   

        
        cv2.imshow('frame', dframe)  
            

In [73]:
h = HoughTransform()
h.run(fromvideo=True)

slope 1.652
slope 0.185
slope 0.296
slope 0.256
slope 0.321
slope 0.205
slope 0.043
slope -0.092
slope -0.027
slope 0.043
slope 0.189
slope 0.322
slope 0.322
slope 0.143
slope 0.242
slope 0.143
slope 0.043
slope 0.053
slope 0.11
slope 0.11
slope 0.11
slope 0.317
slope 0.043
slope 0.043
slope 0.333
slope 0.613
slope 0.613
slope 0.22
slope 0.22
slope 0.185
slope 0.298
slope 0.143
slope 0.348
slope 0.256
slope 0.256
slope 0.295
slope 0.295
slope 0.121
slope -0.069
slope 0.16
slope 0.242
slope 0.018
slope 0.018
slope 0.018
slope 0.042
slope 0.205
slope 0.317
slope 0.069
slope 0.0
slope -0.023
slope -0.023
slope 0.083
slope 0.083
slope 0.156
slope 0.231
slope 0.287
slope 0.209
slope -0.009
slope 0.043
slope 0.016
slope 0.173
slope 0.016
slope -0.049
slope 0.016
slope 0.173
slope 0.136
slope 0.0
slope 0.058
slope 0.087
slope 0.215
slope -0.146
slope -0.054
slope -0.036
slope -0.133
slope -0.036
slope -0.433
slope 0.003
slope -0.089
slope -0.008
slope 0.174
slope -0.027
slope 0.182
slope 0.05

slope -0.85
slope -0.85
slope -0.814
slope -0.95
slope -1.057
slope -0.887
slope -0.85
slope -0.85
slope -1.038
slope -1.1
slope -1.062
slope -1.123
slope -1.1
slope -1.022
slope -1.153
slope -1.123
slope -1.035
slope -1.062
slope -1.1
slope -1.083
slope -1.05
slope -1.25
slope -1.117
slope -1.029
slope -1.211
slope -1.062
slope -1.123
slope -1.211
slope -1.153
slope -0.85
slope -0.85
slope -0.805
slope -0.805
slope -0.89
slope -0.85
slope -0.805
slope -0.85
slope -0.85
slope -0.85
slope -0.85
slope -0.892
slope -0.886
slope -0.888
slope -0.85
slope -0.85
slope -0.85
slope -0.85
slope -0.85
slope -0.85
slope -0.85
slope -0.85
slope -0.85
slope -0.85
slope -0.898
slope -0.85
slope -0.85
slope -0.85
slope -0.85
slope -0.85
slope -0.802
slope -0.85
slope -0.802
slope -0.85
slope -0.85
slope -0.895
slope -0.85
slope -0.895
slope -0.805
slope -1.214
slope -1.714
slope -1.267
slope -1.154
slope -1.017
slope -0.558
slope -0.589
slope -0.893
slope -0.633
slope -0.85
slope -0.35
slope -0.3
slop

slope -0.115
slope 0.062
slope 0.062
slope 0.128
slope 0.07
slope 0.157
slope -0.098
slope -0.154
slope 0.087
slope 0.0
slope 0.223
slope 0.157
slope -0.154
slope 0.366
slope -0.038
slope -0.115
slope 0.128
slope 0.045
slope 0.022
slope 0.022
slope 0.271
slope -0.038
slope -0.168
slope 0.045
slope 0.114
slope 0.087
slope -0.038
slope -0.094
slope -0.131
slope 0.128
slope 0.412
slope -0.038
slope 0.201
slope -0.038
slope 0.114
slope -0.038
slope 0.114
slope 0.157
slope 0.189
slope 0.128
slope 0.045
slope 0.319
slope 0.114
slope 0.114
slope 0.189
slope 0.234
slope 0.234
slope 0.362
slope 0.045
slope 0.201
slope 0.28
slope 0.366
slope 0.114
slope 0.234
slope 0.128
slope 0.045
slope 0.412
slope 0.022
slope -0.038
slope 0.128
slope -0.038
slope 0.128
slope -0.058
slope 0.412
slope -0.058
slope 0.362
slope 0.234
slope 0.157
slope 0.062
slope 0.022
slope 0.114
slope 0.362
slope 0.28
slope -0.018
slope 0.201
slope 0.114
slope 0.234
slope 0.003
slope 0.412
slope 0.366
slope 0.128
slope 0.045
sl